In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from pydrake.all import (AddContactMaterial, AddMultibodyPlant, CoulombFriction,
                         DeformableBodyConfig, DiagramBuilder, Filament,
                         GeometryInstance, IllustrationProperties, Meshcat,
                         MeshcatVisualizer, MeshcatVisualizerParams,
                         MultibodyPlantConfig, ProximityProperties, Rgba,
                         RigidTransform, RotationMatrix, Simulator, Sphere)

In [ ]:
meshcat = Meshcat()
meshcat.SetProperty("/Grid", "visible", False)
meshcat.SetProperty("/Axes", "visible", False)
meshcat.SetProperty("/Background", "visible", False)

In [ ]:
youngs_modulus = 1e7
mass_density = 500
radius = 0.01
cross_section_diameter = 0.0025
segments = 20
num_rings = 100

In [ ]:
builder = DiagramBuilder()

plant_config = MultibodyPlantConfig()
plant_config.time_step = 8e-4

plant, scene_graph = AddMultibodyPlant(plant_config, builder)
deformable_model = plant.mutable_deformable_model()

theta = np.arange(0, 2*np.pi, 2*np.pi/segments)
node_positions = np.vstack([
    np.cos(theta) * radius,
    np.zeros_like(theta),
    np.sin(theta) * radius,
])
ring = Filament(
    closed=True,
    node_pos=node_positions,
    cross_section=Filament.CircularCrossSection(cross_section_diameter)
)

illus_props = IllustrationProperties()
illus_props.AddProperty("phong", "diffuse", Rgba(0.5,0.5,0.5,1.0))

proximity_props = ProximityProperties()
AddContactMaterial(
    properties=proximity_props,
    friction=CoulombFriction(0.1, 0.1),
    dissipation=50,
    point_stiffness=1e4,
)

config = DeformableBodyConfig()
config.set_youngs_modulus(youngs_modulus)
config.set_poissons_ratio(0.4)
config.set_mass_density(mass_density)
config.set_mass_damping_coefficient(10)

body_ids = []
for i in range(num_rings):
    geometry = GeometryInstance(RigidTransform(
        RotationMatrix.MakeZRotation(np.pi / 2 * i),
        [0, 0, -(radius + cross_section_diameter) * i]
    ), ring, f"ring{i}")
    geometry.set_proximity_properties(proximity_props)

    if i == 0:
        color = Rgba(0.0, 0.0, 1.0, 1)
    elif i % 2 == 0:
        color = Rgba(0.6, 1.0, 0.6, 1)
    else:
        color = Rgba(1.0, 0.6, 0.6, 1)
    illus_props.UpdateProperty("phong", "diffuse", color)
    geometry.set_illustration_properties(illus_props)

    unused_resolution_hint = 9999
    body_id = deformable_model.RegisterDeformableBody(
        geometry, config, unused_resolution_hint)
    body_ids.append(body_id)

deformable_model.SetWallBoundaryCondition(body_ids[0], [0,0,0], [0,0,1])

dummy = plant.AddRigidBody("dummy")
plant.WeldFrames(plant.world_frame(), dummy.body_frame())
plant.RegisterVisualGeometry(dummy, RigidTransform(), Sphere(0.01), "dummy", np.zeros(4))

plant.Finalize()

# Add Meshcat visualizer
visualizer = MeshcatVisualizer.AddToBuilder(
    builder, scene_graph, meshcat, MeshcatVisualizerParams(publish_period=1/100)
)

diagram = builder.Build()

simulator = Simulator(diagram)

visualizer.StartRecording()
simulator.AdvanceTo(0.8 + 1e-5)
visualizer.StopRecording()
visualizer.PublishRecording()